In [53]:
import os
import socket
from datetime import date,datetime,timedelta
import time
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F
import socket

#获取计算机名称
hostname=socket.gethostname()
#获取本机IP
ip=socket.gethostbyname(hostname)
print(ip)

172.20.2.140


In [54]:

# 设置pyspark 提交job的环境变量
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /root/dependencies/xgboost4j_3.0-1.3.0-0.1.0.jar,/root/dependencies/xgboost4j-spark_3.0-1.3.0-0.1.0.jar,/root/dependencies/clickhouse-native-jdbc-2.2-stable.jar,/root/dependencies/clickhouse4j-1.4.4.jar,/root/dependencies/joda-time-2.9.9.jar pyspark-shell'

# Create Spark config for our Kubernetes based cluster manager
sparkConf = SparkConf()
#### K8s调度 配置参数
sparkConf.setMaster("k8s://https://kubernetes.default.svc.cluster.local:443")
sparkConf.set("spark.kubernetes.container.image", "harbor.aift.ftwhale.com/library/spark/pyspark:v3.0.1-aift")
sparkConf.set("spark.kubernetes.container.image.pullPolicy", "Always")
sparkConf.set("spark.kubernetes.namespace", "default")
sparkConf.set("spark.kubernetes.authenticate.driver.serviceAccountName", "spark")
sparkConf.set("spark.kubernetes.authenticate.serviceAccountName", "spark")
sparkConf.set("spark.kubernetes.pyspark.pythonVersion", "3")
#### 配置executor的volume挂载
sparkConf.set("spark.kubernetes.executor.volumes.persistentVolumeClaim.heketi-jupyter-pvc.mount.path","/root/data")
sparkConf.set("spark.kubernetes.executor.volumes.persistentVolumeClaim.heketi-jupyter-pvc.readOnly","false")
sparkConf.set("spark.kubernetes.executor.volumes.persistentVolumeClaim.heketi-jupyter-pvc.options.claimName","heketi-jupyter-pvc")

#### spark driver 配置                                                          ####################################获取本地ip放进去
# sparkConf.set("spark.driver.host", "{}".format(ip)) # 区别于其他人的jupyter（jupyter的域名） 必改！
sparkConf.set("spark.driver.host", "172.20.2.140")
sparkConf.set("spark.driver.port", "29413")
sparkConf.set("spark.driver.extraClassPath", "/root/dependencies/xgboost4j_3.0-1.3.0-0.1.0.jar:/root/dependencies/xgboost4j-spark_3.0-1.3.0-0.1.0.jar")
sparkConf.set("spark.driver.memory", "8g")
sparkConf.set("spark.driver.cores", "1")
#### spark driver 调优
sparkConf.set("spark.driver.allowMultipleContexts", "true")
sparkConf.set("spark.driver.maxResultSize", "7g")            #################################################最好不大于8g

#### spark executor 配置
sparkConf.set("spark.executor.extraClassPath", "/root/dependencies/xgboost4j_3.0-1.3.0-0.1.0.jar:/root/dependencies/xgboost4j-spark_3.0-1.3.0-0.1.0.jar")
sparkConf.set("spark.executor.instances", "4") # 时间计算实例的数量
sparkConf.set("spark.executor.cores", "2") # 每个计算实例的cpu
sparkConf.set("spark.executor.memory", "8g") # 每个计算实例的内存
#### spark executor 调优


#### spark tasks 调优
sparkConf.set("spark.task.cpus", 1)
sparkConf.setAppName("huzize-AP1816") # 区别于其他人提交任务的名称 必改！####################################改名字

#### spark sql 配置
sparkConf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
sparkConf.set("spark.sql.inMemoryColumnarStorage.batchSize",50000)
sparkConf.set("spark.sql.autoBroadcastJoinThreshold", 1024*1024*512)
sparkConf.set("spark.sql.shuffle.partitions", 100)
sparkConf.set("spark.sql.parquet.columnarReaderBatchSize", 5000000)
sparkConf.set("spark.sql.files.maxPartitionBytes", 1024*1024*512)
sparkConf.set("spark.sql.broadcastTimeout", 10*60*1000)
sparkConf.set("spark.sql.adaptive.enabled", "true")

#### spark 其他配置参数
sparkConf.set("spark.rpc.message.maxSize", 2040)
# sparkConf.set("spark.submit.pyFiles", "/root/dependencies/spark3-samples.zip,/root/dependencies/xgboost4j-spark_3.0-1.3.0-0.1.0.jar")
# sparkConf.set("spark.submit.pyFiles", "/root/data/samples.zip,/root/data/xgboost4j-spark_3.0-1.0.0-0.1.0.jar")

# Initialize our Spark cluster, this will actually
# generate the worker nodes.
spark = SparkSession.builder.config(conf=sparkConf).getOrCreate()
spark

In [55]:
def write_data_toClickHouse(target_sdf,i,table):
    try:
        target_sdf.write.mode("append").format("jdbc") \
            .option("driver", "com.github.housepower.jdbc.ClickHouseDriver") \
            .option("url", "jdbc:clickhouse://chi-ftabc-clickhouse-{}-0.default.svc.cluster.local:9000".format(i)) \
            .option("dbtable", table) \
            .option("batchsize",target_sdf.count())\
            .option("user", "ftabc") \
            .option("password", "aihub@2020") \
            .option("numPartitions", "1") \
            .option("isolationLevel", "NONE") \
            .option("truncate", "true") \
            .save()
    except Exception as e:
        message = str(e).split('\n')[1]
        ignore_message = ': java.sql.SQLFeatureNotSupportedException'      ########################除了这个错误没关系外 其他的都有问题
        if(message == ignore_message):
            print('分片'+str(i)+'插入数据成功')
        else:
            print('分片'+str(i)+'插入数据不成功')
    else:
        print('分片'+str(i)+'已成功插入数据')

In [56]:
def read_data_ClickHouse(sdf_name,i,dbtable):
    sdf_name = spark.read.format("jdbc") \
            .option("driver", "cc.blynk.clickhouse.ClickHouseDriver") \
            .option("url", "jdbc:clickhouse://chi-ftabc-clickhouse-{}-0.default.svc.cluster.local:8123".format(i)) \
            .option("dbtable", dbtable) \
            .option("user", "ftabc") \
            .option("password", "aihub@2020") \
            .load()
    return sdf_name

----

- 得到ods_user表的id列表

In [57]:
sdf_ods_user = spark.read.parquet("/root/data/guiyang.yang/YGY/ClickHouse/data/ods_user.parquet")

In [58]:
list_id_collect = sdf_ods_user.select('id').collect()
list_id = [row[0] for row in list_id_collect]

In [59]:
list_gender = [0,1,2]
list_user_level = [0,1,2,3,4,5]
list_certificate = [110001,110002,110003,110004,110005,110006,110007,110008,110009,110011,\
                    110013,110015,110017,110018,110019,110020,110021,110022,110023,110024,\
                    110025,110026,110027,110028,110029,110030,110031,110033,110034,110035,\
                    110037,110047,110055,110057,119999]
list_nation = [0,1,2,3,4,5,6,7,8,10,\
               11,12,13,14,15,16,17,18,19,20,\
               21,22,23,24,25,26,27,28,29,30,\
               31,32,33,34,35,36,37,38,39,40,\
               41,42,43,44,45,46,47,48,49,50,\
               51,52,53,54,55,56,57,98,99]
list_education = [0,1,2,3,4,5,6,7,8,10,\
               11,12,13,14,15,16,17,18,19,20,\
               21,22,23,24,25,26,27,28,29,30,\
               31,32,33,34,35,36,37,38,39,40,\
               41,42,43,44,45,46,47,48,49,50,\
               51,52,53,54,55,56,57,58,59,60,\
               61,62,63,64,65,66,67,68,69,70,\
               71,72,73,74,75,76,77,78,79,80,\
               81,82,83,84,85,86,87,88]
list_populatioin = [0,1,2,3,4,5,6,7,8,10,\
                   11,19,20,26,30,42,44,50,52,64,\
                   66,70,78,80,90,98,99,103,134,148,\
                   151,184,198,203,218,219,231,234,240,247,\
                   248,250,251]
list_job_level = [0,1,2,3,4,5,6,7,8,10,\
                  11,12,22,32,37,42,51,52,53,57,\
                  58,60,63,67,72,75,78,79,80,82,\
                  93,98,99]
list_relationship_type = [0,1,2]
list_marriage = [0,1,2,3,4]
list_inline_customer = ['0','1']

- 将列表中的数据转化为一个dataframe，然后与ods_user连接

In [60]:
import os
import time
import pandas as pd

from faker import Faker
from datetime import date,datetime,timedelta
import random
import numpy as np

import json
import string
fake = Faker()             #############生成各种各样的伪数据

In [61]:
class ods_userDataGen:
    def __init__(self):
        self.data_dict = {}
    def fake_data(self, a, list_id):
        pt_profile = fake.profile()
        self.data_dict['c_id'] = list_id[a]
        self.data_dict['gender'] = np.random.choice(list_gender)
        self.data_dict['user_level'] = np.random.choice(list_user_level)
        self.data_dict['certificate'] = np.random.choice(list_certificate) 
        self.data_dict['nation'] = np.random.choice(list_nation)
        self.data_dict['education'] = np.random.choice(list_education)
        self.data_dict['population'] = np.random.choice(list_populatioin)
        self.data_dict['job_level'] = np.random.choice(list_job_level)
        self.data_dict['relationship_type'] = np.random.choice(list_relationship_type)
        self.data_dict['marriage']= np.random.choice(list_marriage)
        self.data_dict['inline_customer']= random.choice(list_inline_customer)
        

    def get_data_series(self):
        return pd.Series(self.data_dict)

    def get_data_dict(self):
        return self.data_dict


rst_list = []
for b in range(len(list_id)):
    pt_data = ods_userDataGen()
    pt_data.fake_data(b,list_id)
    rst_list.append(pt_data.get_data_dict())

df = pd.DataFrame(rst_list)

In [62]:
df

,c_id,gender,user_level,certificate,nation,education,population,job_level,relationship_type,marriage,inline_customer
0,1096,1,1,110003,34,77,99,4,2,3,0
1,1087,2,3,110020,37,67,247,80,1,0,0
2,2090,2,4,110018,26,16,42,11,0,0,0
3,4474,0,2,110028,4,74,11,82,2,0,0
4,5803,2,2,110017,40,58,4,12,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...
9995,2688,0,3,110029,49,82,148,78,1,2,1
9996,3127,0,0,110007,1,17,80,58,0,3,1
9997,5289,2,0,110007,43,83,240,6,0,4,1
9998,7786,2,0,110027,37,86,7,80,0,1,0


In [63]:
sdf_correct = spark.createDataFrame(df)

In [64]:
sdf_correct

DataFrame[c_id: bigint, gender: bigint, user_level: bigint, certificate: bigint, nation: bigint, education: bigint, population: bigint, job_level: bigint, relationship_type: bigint, marriage: bigint, inline_customer: string]

In [65]:
sdf_error = sdf_ods_user.drop('gender','user_level','certificate','nation','education',\
                              'population','job_level','relationship_type','marriage','inline_customer')

In [66]:
sdf_error

DataFrame[bal_date: date, user_id: string, id: bigint, age: int, age_range: int, income_year: float, aum: float, aum_range: int, job: string, birthday: date, fund_bal: float, country: int, account_opening_date: date, ifib_bal: float, sub_branch: string, fa_bal: float, residence: string, second_level_branch: string, account_opening_institution: bigint, place_of_residence: string, load_bal: float, bond_bal: float, first_level_branch: string, deposit: float, postal_code: string, oth_inv_bal: float]

In [67]:
sdf_ods_user_right = sdf_error.join(sdf_correct,sdf_error.id==sdf_correct.c_id,'inner').drop('c_id')

In [68]:
sdf_ods_user_right

DataFrame[bal_date: date, user_id: string, id: bigint, age: int, age_range: int, income_year: float, aum: float, aum_range: int, job: string, birthday: date, fund_bal: float, country: int, account_opening_date: date, ifib_bal: float, sub_branch: string, fa_bal: float, residence: string, second_level_branch: string, account_opening_institution: bigint, place_of_residence: string, load_bal: float, bond_bal: float, first_level_branch: string, deposit: float, postal_code: string, oth_inv_bal: float, gender: bigint, user_level: bigint, certificate: bigint, nation: bigint, education: bigint, population: bigint, job_level: bigint, relationship_type: bigint, marriage: bigint, inline_customer: string]

In [69]:
sdf_ods_user_right.count()

10000

- 保存一份ods_user的parquet文件，备份到自己的目录

In [70]:
# 将spark.dataframe保存为parquet文件
sdf_ods_user.write.parquet("/root/data/huzize/ods_user.parquet",mode='overwrite')



In [71]:
# 读取parquet文件为spark.dataframe
sdf_spark = spark.read.parquet("/root/data/huzize/ods_user.parquet")

In [72]:
sdf_spark

DataFrame[bal_date: date, user_id: string, id: bigint, age: int, age_range: int, gender: int, education: int, income_year: float, aum: float, aum_range: int, job_level: int, job: string, marriage: int, birthday: date, fund_bal: float, country: int, account_opening_date: date, nation: int, certificate: bigint, ifib_bal: float, relationship_type: int, sub_branch: string, fa_bal: float, residence: string, second_level_branch: string, account_opening_institution: bigint, place_of_residence: string, population: int, load_bal: float, inline_customer: string, bond_bal: float, first_level_branch: string, deposit: float, user_level: int, postal_code: string, oth_inv_bal: float]

- 清空当前ods_user表的数据，并插入修正后的数据

In [73]:
from clickhouse_driver import connect
import clickhouse_driver

client = clickhouse_driver.Client(host='chi-ftabc-clickhouse-0-0.default.svc.cluster.local', port='9000', database='ftabcch', user='ftabc', password='aihub@2020')
conn = connect(host='chi-ftabc-clickhouse-1-0.default.svc.cluster.local', port='9000', database='ftabcch', user='ftabc', password='aihub@2021')
cursor = conn.cursor()

In [76]:
# 清空当前clickhouse的ods_user表的数据
# cursor.execute('alter table ods_user on cluster "clickhouse" delete where id is not null')
# cursor.fetchall()
# cursor.execute('''
#     select 
#         '{}'
# '''.format('123'))
# this_two_month = cursor.fetchall()
# print(this_two_month)

- 插入修改后的数据

In [95]:
ods_user1,ods_user2,ods_user3,ods_user4 = sdf_ods_user_right.randomSplit([0.25,0.25,0.25,0.25])

In [98]:
write_data_toClickHouse(ods_user1,0,"ftabcch.ods_user")
write_data_toClickHouse(ods_user2,1,"ftabcch.ods_user")
write_data_toClickHouse(ods_user3,2,"ftabcch.ods_user")
write_data_toClickHouse(ods_user4,3,"ftabcch.ods_user")

分片0插入数据成功
分片1插入数据成功
分片2插入数据成功
分片3插入数据成功


- 保存一份到根目录

In [100]:
sdf_ods_user_right.write.parquet("/root/data/ods_user.parquet",mode='overwrite')

In [101]:
spark.stop()